In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 04:35:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/25 04:35:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark.sql("create database shellida")

DataFrame[]

In [5]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
| shellida|
+---------+



In [6]:
spark.sql("use shellida")

DataFrame[]

In [53]:
salesdf=spark.read.option("header","true").option("inferSchema","true").csv("/home/labuser/Documents/datasets/superstore.csv")

In [24]:
salesdf.show()

+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|   ID|        OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|    CustomerName|    Segment|         City|          State|      Country|PostalCode|Market|      Region|       ProductID|       Category|Sub-Category|         ProductName|           Sales|Quantity|Discount|    Profit|ShippingCost|OrderPriority|
+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|32298| CA-2012-124891|31/07/2012|31/07/2012|      S

In [10]:
salesdf.write.mode("overwrite").saveAsTable("sale")

In [13]:
salesdf.show()

+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|   ID|        OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|    CustomerName|    Segment|         City|          State|      Country|PostalCode|Market|      Region|       ProductID|       Category|Sub-Category|         ProductName|           Sales|Quantity|Discount|    Profit|ShippingCost|OrderPriority|
+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|32298| CA-2012-124891|31/07/2012|31/07/2012|      S

In [15]:
sales_US=salesdf.select("country","state","profit").filter("country='United States'").show()

+-------------+--------------+----------+
|      country|         state|    profit|
+-------------+--------------+----------+
|United States|      New York|  762.1845|
|United States|    California|  1906.485|
|United States|North Carolina|-1862.3124|
|United States|      Virginia|    83.281|
|United States|      Kentucky|  517.4793|
|United States|      Illinois|   341.994|
|United States|    California|  363.9048|
|United States|         Texas|   -350.49|
|United States|    California|  135.4068|
|United States|      New York| 1371.9804|
|United States|North Carolina|-3839.9904|
|United States|     Minnesota| 4630.4755|
|United States|      New York|  1143.891|
|United States|    California|   839.986|
|United States|    Washington| 1480.4671|
|United States|       Florida|  327.5922|
|United States|      New York|  2229.024|
|United States|      Virginia|  694.5015|
|United States|       Georgia|  3177.475|
|United States|      Michigan| 2504.2216|
+-------------+--------------+----

In [16]:
salesdf.rdd.getNumPartitions()

2

In [27]:
salesdf.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- OrderID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- ShipDate: string (nullable = true)
 |-- ShipMode: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PostalCode: integer (nullable = true)
 |-- Market: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: string (nullable = true)
 |-- ShippingCost: string (nullable = true)
 |-- OrderPriority: string (nullable = true)



In [18]:
from pyspark.sql.functions import col,sum
sales_US=salesdf.groupBy("Country","state").agg(sum(col("Profit").cast('int')).alias("totSales")).filter("country='United States'").show()

+-------------+--------------+--------+
|      Country|         state|totSales|
+-------------+--------------+--------+
|United States|      Virginia|   17154|
|United States|South Carolina|    1774|
|United States| West Virginia|     186|
|United States|         Maine|     417|
|United States|       Montana|    1852|
|United States|       Alabama|    4645|
|United States|       Wyoming|     100|
|United States|         Idaho|     695|
|United States|    New Jersey|    6916|
|United States|      Arkansas|    3437|
|United States|      Michigan|   21869|
|United States|      Illinois|   -9298|
|United States|          Ohio|  -14490|
|United States|      New York|   65943|
|United States|      Missouri|    6208|
|United States|   Mississippi|    2445|
|United States|      Nebraska|    1421|
|United States|     Minnesota|    9820|
|United States|    Washington|   29900|
|United States|        Kansas|     794|
+-------------+--------------+--------+
only showing top 20 rows



In [19]:
from pyspark.sql.functions import col,sum,avg
sales_US =salesdf.groupBy("Country","state").agg(sum(col("Profit").cast('int')).alias("totSales"),avg(col("Profit").cast('int')).alias("AvgSales")).filter("country='United States'").show()

+-------------+--------------+--------+-------------------+
|      Country|         state|totSales|           AvgSales|
+-------------+--------------+--------+-------------------+
|United States|      Virginia|   17154|  76.58035714285714|
|United States|South Carolina|    1774|  42.23809523809524|
|United States| West Virginia|     186|               46.5|
|United States|         Maine|     417|             52.125|
|United States|       Montana|    1852| 123.46666666666667|
|United States|       Alabama|    4645|  76.14754098360656|
|United States|       Wyoming|     100|              100.0|
|United States|         Idaho|     695| 33.095238095238095|
|United States|    New Jersey|    6916|   53.6124031007752|
|United States|      Arkansas|    3437|  57.28333333333333|
|United States|      Michigan|   21869|   86.0984251968504|
|United States|      Illinois|   -9298|-18.898373983739837|
|United States|          Ohio|  -14490|-30.895522388059703|
|United States|      New York|   65943| 

In [20]:
salesdf.write.mode("overwrite").saveAsTable("sale")

In [22]:
sales4tabledf= spark.read.table("sale")

In [23]:
sales4tabledf.show()

+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|   ID|        OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|    CustomerName|    Segment|         City|          State|      Country|PostalCode|Market|      Region|       ProductID|       Category|Sub-Category|         ProductName|           Sales|Quantity|Discount|    Profit|ShippingCost|OrderPriority|
+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|32298| CA-2012-124891|31/07/2012|31/07/2012|      S

In [29]:
salesdf.select("Region","country").sort(col("region").desc()).orderBy("country",ascending=False).filter("region='Central Asia'").distinct().show()

+------------+-----------+
|      Region|    country|
+------------+-----------+
|Central Asia|      Nepal|
|Central Asia|Afghanistan|
|Central Asia| Bangladesh|
|Central Asia|  Sri Lanka|
|Central Asia|   Pakistan|
|Central Asia|      India|
+------------+-----------+



In [31]:
 spark.sql("select country,region,avg(profit) from sale group by country,region").filter("Country='India'").orderBy("region").show()

+-------+------------+--------------------+
|country|      region|         avg(profit)|
+-------+------------+--------------------+
|  India|Central Asia|0.017684887459807074|
+-------+------------+--------------------+



In [34]:
from pyspark.sql.functions import col, sum,avg

sales_US =salesdf.groupBy("Country","state").agg(sum(col("Profit").cast('int')).alias("totSales"),avg(col("Profit").cast('int')).alias("AvgSales")).filter("country='United States'").show()

+-------------+--------------+--------+-------------------+
|      Country|         state|totSales|           AvgSales|
+-------------+--------------+--------+-------------------+
|United States|      Virginia|   17154|  76.58035714285714|
|United States|South Carolina|    1774|  42.23809523809524|
|United States| West Virginia|     186|               46.5|
|United States|         Maine|     417|             52.125|
|United States|       Montana|    1852| 123.46666666666667|
|United States|       Alabama|    4645|  76.14754098360656|
|United States|       Wyoming|     100|              100.0|
|United States|         Idaho|     695| 33.095238095238095|
|United States|    New Jersey|    6916|   53.6124031007752|
|United States|      Arkansas|    3437|  57.28333333333333|
|United States|      Michigan|   21869|   86.0984251968504|
|United States|      Illinois|   -9298|-18.898373983739837|
|United States|          Ohio|  -14490|-30.895522388059703|
|United States|      New York|   65943| 

In [41]:
sales_withcolumn = salesdf.withColumn("SalesLevel", when(col("profit")>=10000,"High").when(col("profit")<10000,"Average").otherwise("NA"))  

In [42]:
sales_withcolumn.select("country","state","profit","salesLevel").show()

+-------------+---------------+----------+----------+
|      country|          state|    profit|salesLevel|
+-------------+---------------+----------+----------+
|United States|       New York|  762.1845|   Average|
|    Australia|New South Wales|       0.1|   Average|
|    Australia|     Queensland|       0.1|   Average|
|      Germany|         Berlin|       0.1|   Average|
|      Senegal|          Dakar|         0|   Average|
|    Australia|New South Wales|       0.1|   Average|
|  New Zealand|     Wellington|         0|   Average|
|  New Zealand|        Waikato|         0|   Average|
|United States|     California|  1906.485|   Average|
|United States| North Carolina|-1862.3124|   Average|
|United States|       Virginia|    83.281|   Average|
|  Afghanistan|          Kabul|         0|   Average|
| Saudi Arabia|          Jizan|         0|   Average|
|       Brazil|         Parana|         0|   Average|
|        China|   Heilongjiang|         0|   Average|
|       France|  Ile-de-Fran

In [43]:
sales_US

DataFrame[ID: int, OrderID: string, OrderDate: string, ShipDate: string, ShipMode: string, CustomerID: string, CustomerName: string, Segment: string, City: string, State: string, Country: string, PostalCode: int, Market: string, Region: string, ProductID: string, Category: string, Sub-Category: string, ProductName: string, Sales: string, Quantity: string, Discount: string, Profit: string, ShippingCost: string, OrderPriority: string, SalesLevel: string]

In [47]:
sales_US=sales_US.withColumnRenamed("totSales","TotalSales")

In [55]:
from pyspark.sql.functions import lit
sales_US.withColumn("CountryCode",lit("US"))

DataFrame[ID: int, OrderID: string, OrderDate: string, ShipDate: string, ShipMode: string, CustomerID: string, CustomerName: string, Segment: string, City: string, State: string, Country: string, PostalCode: int, Market: string, Region: string, ProductID: string, Category: string, Sub-Category: string, ProductName: string, Sales: string, Quantity: string, Discount: string, Profit: string, ShippingCost: string, OrderPriority: string, CountryCode: string]

In [56]:
sales_US

DataFrame[ID: int, OrderID: string, OrderDate: string, ShipDate: string, ShipMode: string, CustomerID: string, CustomerName: string, Segment: string, City: string, State: string, Country: string, PostalCode: int, Market: string, Region: string, ProductID: string, Category: string, Sub-Category: string, ProductName: string, Sales: string, Quantity: string, Discount: string, Profit: string, ShippingCost: string, OrderPriority: string]

In [58]:
employeedf=spark.read.option("header","true").csv("/home/labuser/Documents/datasets/employee.csv",inferSchema=True)

In [59]:
employeedf.show()

+-----+-----+----+
|empno|ename| sal|
+-----+-----+----+
|  111|  zzz|8000|
|  111|  aaa|8888|
|  121|  bbb|8000|
| NULL|  ccc|9000|
|false|  ddd|6000|
|  555|  eee|7000|
|  765| null|null|
|  666|  fff|8890|
|  aaa| null|null|
| True| null|null|
| true| null|null|
|    a|    b|   c|
|    x| NULL|NULL|
| 1000| null|null|
|  222| NULL|NULL|
+-----+-----+----+



In [60]:
employeedf.dropna().show()

+-----+-----+----+
|empno|ename| sal|
+-----+-----+----+
|  111|  zzz|8000|
|  111|  aaa|8888|
|  121|  bbb|8000|
| NULL|  ccc|9000|
|false|  ddd|6000|
|  555|  eee|7000|
|  666|  fff|8890|
|    a|    b|   c|
|    x| NULL|NULL|
| 1000| null|null|
|  222| NULL|NULL|
+-----+-----+----+



In [62]:
employeedf.na.fill("unknown").show()

+-----+-------+-------+
|empno|  ename|    sal|
+-----+-------+-------+
|  111|    zzz|   8000|
|  111|    aaa|   8888|
|  121|    bbb|   8000|
| NULL|    ccc|   9000|
|false|    ddd|   6000|
|  555|    eee|   7000|
|  765|unknown|unknown|
|  666|    fff|   8890|
|  aaa|unknown|unknown|
| True|unknown|unknown|
| true|unknown|unknown|
|    a|      b|      c|
|    x|   NULL|   NULL|
| 1000|   null|   null|
|  222|   NULL|   NULL|
+-----+-------+-------+



In [63]:
employeedf.na.fill("unknown").na.fill(0).show()

+-----+-------+-------+
|empno|  ename|    sal|
+-----+-------+-------+
|  111|    zzz|   8000|
|  111|    aaa|   8888|
|  121|    bbb|   8000|
| NULL|    ccc|   9000|
|false|    ddd|   6000|
|  555|    eee|   7000|
|  765|unknown|unknown|
|  666|    fff|   8890|
|  aaa|unknown|unknown|
| True|unknown|unknown|
| true|unknown|unknown|
|    a|      b|      c|
|    x|   NULL|   NULL|
| 1000|   null|   null|
|  222|   NULL|   NULL|
+-----+-------+-------+



In [71]:
employeedf.na.fill({'sal':1000,'ename':'unknown'}).dropna().show()

+-----+-------+----+
|empno|  ename| sal|
+-----+-------+----+
|  111|    zzz|8000|
|  111|    aaa|8888|
|  121|    bbb|8000|
| NULL|    ccc|9000|
|false|    ddd|6000|
|  555|    eee|7000|
|  765|unknown|1000|
|  666|    fff|8890|
|  aaa|unknown|1000|
| True|unknown|1000|
| true|unknown|1000|
|    a|      b|   c|
|    x|   NULL|NULL|
| 1000|   null|null|
|  222|   NULL|NULL|
+-----+-------+----+



In [74]:
employeedf.na.fill({'sal':1000,'ename':'unknown'}).dropna().show()

+-----+-------+----+
|empno|  ename| sal|
+-----+-------+----+
|  111|    zzz|8000|
|  111|    aaa|8888|
|  121|    bbb|8000|
| NULL|    ccc|9000|
|false|    ddd|6000|
|  555|    eee|7000|
|  765|unknown|1000|
|  666|    fff|8890|
|  aaa|unknown|1000|
| True|unknown|1000|
| true|unknown|1000|
|    a|      b|   c|
|    x|   NULL|NULL|
| 1000|   null|null|
|  222|   NULL|NULL|
+-----+-------+----+



In [75]:
employeedf.na.replace(8000,9999).show()

+-----+-----+----+
|empno|ename| sal|
+-----+-----+----+
|  111|  zzz|8000|
|  111|  aaa|8888|
|  121|  bbb|8000|
| NULL|  ccc|9000|
|false|  ddd|6000|
|  555|  eee|7000|
|  765| null|null|
|  666|  fff|8890|
|  aaa| null|null|
| True| null|null|
| true| null|null|
|    a|    b|   c|
|    x| NULL|NULL|
| 1000| null|null|
|  222| NULL|NULL|
+-----+-----+----+



In [77]:
employeedf.na.replace("NULL",None).na.fill({'sal':1000,'ename':'unknown'}).dropna().show()

+-----+-------+----+
|empno|  ename| sal|
+-----+-------+----+
|  111|    zzz|8000|
|  111|    aaa|8888|
|  121|    bbb|8000|
|false|    ddd|6000|
|  555|    eee|7000|
|  765|unknown|1000|
|  666|    fff|8890|
|  aaa|unknown|1000|
| True|unknown|1000|
| true|unknown|1000|
|    a|      b|   c|
|    x|unknown|1000|
| 1000|   null|null|
|  222|unknown|1000|
+-----+-------+----+

